In [1]:
# Import my code
#from tensorfactorization.poisson import *
from tensorfactorization.multiplicative import *

In [2]:
import tensorly as tl
import numpy as np
import math
import time
import random
import matplotlib.pyplot as plt
from skimage import data

from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D

from dataclasses import dataclass, field
from typing import List

import copy # import copy so that we can do deep copys

import pickle # use pickle to save results to disk

In [19]:
def tensor_factorization_cp_poisson(X, F, error=1e-6, max_iter=500, detailed=False, verbose=False):
    """
    This function uses a multiplicative method to calculate a nonnegative tensor decomposition
    
    Args:
      X: The tensor of dimension N we want to decompose. X \in \RR^{I_1 x ... x I_N}
      F: The order of the apporximation
      error: stops iteration if difference between X and approximation with decomposition changes less then this
      max_iter: maximum number of iterations
      detailed: if false, function returns only G and the As. if true returns also all errors found during calculation 
      verbose: If True, prints additional information
    
    Returns:
      A list of tensors approximating X 
    """
    
    N = X.ndim # get dimension of X
    X_shape = X.shape
    norm_X = tl.norm(X)
    # initialize A_j with random positive values
    A_ns = []
    for i in range(N):
        # we use random.random_tensor as it returns a tensor
        A_ns.append(tl.random.random_tensor((X_shape[i], F), **tl.context(X)))
    
    # the reconstruction error
    approximated_X = defactorizing_CP(A_ns, X_shape)
        
    RE = [tl.norm(X-approximated_X)/norm_X]
    for _ in range(max_iter):
        for n in range(N):
            start = time.time()
            
            khatri_rao_product = tl.tenalg.khatri_rao(A_ns, skip_matrix=n)
            approximated_X_unfolded_n = approximated_X #tl.matmul(A_ns[n], tl.transpose(khatri_rao_product))
            
            # regular * does componentwise multiplication
            step_size = 1.0 # TODO add step size calculation
            A_ns[n] = A_ns[n] * np.exp(-step_size * tl.matmul( tl.base.unfold(X, n) / approximated_X_unfolded_n , khatri_rao_product )  ) 
            
            end = time.time()
            if verbose:
                print("Current index: " + str(n))
                print("Calculculation time: " + str(end - start))
                print("new A_ns[n]:")
                print(A_ns[n])
                
            
        # the reconstruction error
        approximated_X = defactorizing_CP(A_ns, X_shape)
        RE.append(tl.norm(X-approximated_X)/norm_X)

        if verbose:
            print("current apporximation error is: " + str(RE[-1]))
            print("approximation is:")
            print(approximated_X)
        
        # check if we have converged
        if abs(RE[-1] - RE[-2]) < error:
            break

    # TODO I think we can skip this for now
    """
    # Rescale the A_ns
    # TODO there should be a smarter way of calculating K
    K = []
    for n in range(N):
        K_j = []
        for a in range(F):
            sum = 0
            for i in range(X_shape[n]):
                sum += A_ns[n][i, a]**2
            K_j.append(math.sqrt(sum))
        K.append(K_j)
    K = tl.tensor(K, **tl.context(X))
    for n in range(N):
        for a in range(F):
            A_ns[n][:, a] = A_ns[n][:, a] * math.pow(tl.prod(K[:, a]), 1.0/N) / K[n, a]
    """

    if detailed:
        return A_ns, RE, approximated_X
    return A_ns

In [22]:
A = tl.random.random_cp((4, 4), 2, full=True)
A = tl.eye(4)
print(A)
tensor_factorization_cp_poisson(A, 2, max_iter=10, detailed=True, verbose=True)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Current index: 0
Calculculation time: 0.0
new A_ns[n]:
[[0.06389362 0.60250317]
 [0.79567616 0.7675471 ]
 [0.29278575 0.4497095 ]
 [0.12987373 0.59262657]]
Current index: 1
Calculculation time: 0.0
new A_ns[n]:
[[0.32309637 0.49493032]
 [0.12292756 0.81484344]
 [0.63915008 0.91914947]
 [0.19805367 0.68822305]]
current apporximation error is: 1.152051736644565
approximation is:
[[0.31884089 0.49880005 0.59462809 0.42731094]
 [0.63696241 0.72324124 1.21404698 0.68583019]
 [0.31717288 0.40243428 0.60048429 0.36748774]
 [0.33527059 0.49886293 0.6277212  0.43358123]]
Current index: 0
Calculculation time: 0.0
new A_ns[n]:
[[0.06324943 0.59322285]
 [0.79432491 0.75894802]
 [0.28968589 0.4428783 ]
 [0.12928184 0.58329407]]
Current index: 1
Calculculation time: 0.0
new A_ns[n]:
[[0.32245607 0.48580698]
 [0.12158485 0.80633742]
 [0.63607411 0.91239536]
 [0.19746401 0.67902643]]
current apporximation error is: 1.1387807045691507
approxi

([array([[0.05758606, 0.51953212],
         [0.78312816, 0.69055911],
         [0.26433443, 0.38881622],
         [0.12407586, 0.50914525]]),
  array([[0.31683216, 0.41345688],
         [0.11046488, 0.73872208],
         [0.61093098, 0.85897359],
         [0.19228008, 0.60602591]])],
 [1.1658429956179934,
  1.152051736644565,
  1.1387807045691507,
  1.1260275386742598,
  1.1137893864354562,
  1.1020628873449847,
  1.0908441591153235,
  1.0801287865323277,
  1.0699118132050558,
  1.0601877364304357,
  1.0509505053559531],
 array([[0.23304925, 0.39015108, 0.48144548, 0.32592258],
        [0.5336366 , 0.59663942, 1.07160929, 0.56907666],
        [0.24450839, 0.3164268 , 0.49547296, 0.28645895],
        [0.24982083, 0.38982286, 0.51314411, 0.33241253]]))